# Logging 

> Some notes regarding logging in Python 

## References 

- [Logging in Python: A Comprehensive Guide to Mastering It](https://www.youtube.com/watch?v=gCD-Wsb_Iw4)
- [Modern Python logging (very good)](https://www.youtube.com/watch?v=9L77QExPmI0)
- [Another course on logging (pretty good)](https://www.youtube.com/watch?v=b4Ms4wxJuPg)
- [`loguru` python library](https://www.youtube.com/watch?v=gSc1oHcwkE4)
- [How to create python decorators (e.g., timing module)](https://www.youtube.com/watch?v=r7Dtus7N4pI)

Python stadndard library documentation on `logging` module: 
- [Logging Cookbook](https://docs.python.org/3/howto/logging-cookbook.html)
- [logging-levels](https://docs.python.org/3/howto/logging.html#logging-levels)
- [configuring-logging](https://docs.python.org/3/howto/logging.html#configuring-logging)
- [logging-config-api](https://docs.python.org/3/library/logging.config.html#logging-config-api)
- [logrecord-attributes](https://docs.python.org/3/library/logging.html#logrecord-attributes)
- [NullHandler](https://docs.python.org/3/library/logging.handlers.html#logging.NullHandler)
- [library-config](https://docs.python.org/3/howto/logging.html#library-config)
- [user-defined-objects](https://docs.python.org/3/library/logging.config.html#user-defined-objects)

To start logging we can simply do

In [1]:
import logging 

logging.basicConfig(level=logging.CRITICAL, filename="./test.log", filemode="w", encoding ="utf-8", format="%(levelname)s-%(name)s-%(message)s") 

logging.critical("critical") 
logging.error("error") 
logging.warning("warning")
logging.info("info") 
logging.debug("Its debug")

The line above configures `root` logger and outputs messages of varied level of severity. 

Levels typically described by the following table: 


|  Level   |  Code  | Description |
|----------|--------|-------------|
| CRITICAL |  50    | Errors that cause application failure, such as a crucial database being unavailable |
| ERROR    |  40    | Handling errors that affect the application's opearation, such as an HTTP 500 error, but allow app to continue working | 
| WARNING  |  30    | Non-critical issues tha require attention. such as depricated code usage or low disk space |
| INFO     |  20    | Informative messages such as user authentication message or version info |
| DEBUG    |  10    | Debugging messages that provide extra information for developers during developmnet or testing |
| NOTSET   |  0     | ...








Loggers have the following components: 
- Handlers
- Formatters
- Filters

![](./python_logging2.png)

![](./python_logging3.png)

To add them we can do the following: 

In [ ]:
import logging
import auxiliary_module

# create logger with 'spam_application'
logger = logging.getLogger('spam_application')
logger.setLevel(logging.DEBUG)

# create file handler which logs even debug messages
fh = logging.FileHandler('spam.log')
fh.setLevel(logging.DEBUG)

# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

logger.info('creating an instance of auxiliary_module.Auxiliary')
a = auxiliary_module.Auxiliary()
logger.info('created an instance of auxiliary_module.Auxiliary')
logger.info('calling auxiliary_module.Auxiliary.do_something')

a.do_something()

logger.info('finished auxiliary_module.Auxiliary.do_something')
logger.info('calling auxiliary_module.some_function()')
auxiliary_module.some_function()
logger.info('done with auxiliary_module.some_function()')

`auxiliary_module.py`

In [ ]:
import logging

# create logger
module_logger = logging.getLogger('spam_application.auxiliary')

class Auxiliary:
    def __init__(self):
        self.logger = logging.getLogger('spam_application.auxiliary.Auxiliary')
        self.logger.info('creating an instance of Auxiliary')

    def do_something(self):
        self.logger.info('doing something')
        a = 1 + 1
        self.logger.info('done doing something')

def some_function():
    module_logger.info('received a call to "some_function"')

Best Practices for Logging: 

Tip #1: 
use `dictConfig` instead of `addHandler` etc. 

Tip #2: 
Put all handlers/filters on the root logger (unless you specifically need not to). 

Tip #3: 
Do not use `root` logger in the code.  

Tip #4: |
If we have a small to mideum size application then a single non-root logger is sufficient. If we have a very large application, then make one non-root logger for each major sub component (don't `getLogger(__name__)` in every file). You do not need logger for EVERY FILE because these are globals which leave for the entire duration of your application. 

Tip #5: 
```
logging_config = {
'version': 1, #<= this is required and the only valid value is 1 (this is so they can change everything in the future without breaking the old code)
'disable_existing_loggers': False,
'formatters': {
        'simple': {
            'format': "%(levelname)s: %(message)s"
        }
    },
'handlers': {
    'stdout': {
        "class": "logging.StreamHandler",
        "formatter": "simple",
        "stream": "ext://sys.stdout"
        }
    },
"loggers": {
    "root": {"level": "DEBUG", "handlers": ["stdout"]}
    }
}
```

Tip #6: 
Store the configuration file in YAML or JSON format. 

Tip #6: 
Use ISO-8601 Timestamps format to log in dates. 
```
"formatters": { 
    "detailed": {
        "format": "[%(levelname)s|%(module)s|L%(lineno)d] %(asctime): %(message)", 
        "datefmt" : "%Y-%m-%dT%H:%M:%S%z"
    }
}
```

Tip #7: 
Store persistent logs in JSON format (so they can be parsed easily later on). 

Tip #10: 
For library code __*do not configure*__ logging. You can still use logging (create loggers, log messages and other important events), just don't configure it with dictConfig and any other config. Let applications do the configuring. If the user does not configure logging the _default_ and _expected_ behavior is that warnings and above will be printed to __standard error__. 

If the user of the library does configure logging then do not interfere with what they want by adding handlers, formatter or other things of which they are not aware of. 


## Meeting 2024-10-04

- Which variables that wont be in the output product would be interesting/required for the log? For example, in data ingestion we throw away samples that are out of bounds of the grid. 
- What was thrown away during quality control
- I guess some of the things to be logged will also be required in the output metadata. 
- Time taken for various things is interesting. 
- Memory / CPU
- Probably some things to do with how the antenna pattern was processed.
- I guess it should log all the configuration file relevant parameters, so the user can check that the correct things are being done
- The end format for log file (`text`, `json` etc.), the naming convention (timestamp; should it be rotated?)

In [ ]:
import logging 
from logging.config import dictConfig 

config = {
    'version': 1, 
    "disable_existing_loggers": False,
    'formatters': {
        'standard': {
            'format': '%(levelname)-10s:%(message)s'
        },
    }, 
    'handlers': {
        'default': {
            'level': 'DEBUG', 
            'formatter': 'standard', 
            'class': 'logging.StreamHandler', 
            "stream": "ext://sys.stdout"
        }, 
        # handler to log stuff into a file
        'file': {
            'level': 'WARNING', 
            'formatter': 'standard', 
            'class': 'logging.FileHandler', 
            'filename': 'dict_test.log', 
            'mode': 'w'
        }
    }, 
    'loggers': {
        "": {
             'level': 'DEBUG', 
            'handlers': ['default', 'file'], 
            'propagate': True 
        }, 
        "richard": {
             'level': 'INFO', 
            'handlers': ['default'], 
            'propagate': False 
        }
    }
}

dictConfig(config)




mylogger = logging.getLogger("richard")  

mylogger.critical("critical") 
mylogger.error("error") 
mylogger.warning("warning")
mylogger.info("info") 
mylogger.debug("Its debug")

In [ ]:
import logging 

def filter_start_x(record: logging.LogRecord): 

    # print(record.__dict__)

    if record.msg.startswith("x"): 
        return False 
    
    return True 

logger  = logging.getLogger() 

logger.warning("This is message 1")

logger.addFilter(filter_start_x)

logger.warning("This is message 2") 

logger.warning("x This is message 2") 


> The below exaple is artificial and in real world scenario we shuld use `contextvars` module of python. 

In [ ]:
import logging 

class ContextFilter(logging.Filter): 

    def filter(self, record: logging.LogRecord): 
        ...


logging.basicConfig(level=logging.DEBUG, format="%(levelname)s - %(name)s - %(message)s") 

logger_a = logging.getLogger("a.b")

ctx_filter = ContextFilter() 

logger_a.addFilter() 

logger_a.warning("API bookings")

The following YouTube videos were very useful: 
- [Logging in Python: A Comprehensive Guide to Mastering It](https://www.youtube.com/watch?v=gCD-Wsb_Iw4)
- [Modern Python logging](https://www.youtube.com/watch?v=9L77QExPmI0)
- [Another course on logging (pretty good)](https://www.youtube.com/watch?v=b4Ms4wxJuPg)
- [`loguru` python library](https://www.youtube.com/watch?v=gSc1oHcwkE4)
- [How to create python decorators (e.g., timing module)](https://www.youtube.com/watch?v=r7Dtus7N4pI)

Tip #1: 
use `dictConfig` instead of `addHandler` etc. 

Tip #2: 
Put all handlers/filters on the root logger (unless you specifically need not to). 

Tip #3: 
Do not use `root` logger in the code.  

Tip #4: 
If we have a small to mideum size application then a single non-root logger is sufficient. If we have a very large application, then make one non-root logger for each major sub component (don't `getLogger(__name__)` in every file). You do not need logger for EVERY FILE because these are globals which leave for the entire duration of your application. 

Tip #5: 
```
logging_config = {
'version': 1, #<= this is required and the only valid value is 1 (this is so they can change everything in the future without breaking the old code)
'disable_existing_loggers': False,
'formatters': {
        'simple': {
            'format': "%(levelname)s: %(message)s"
        }
    },
'handlers': {
    'stdout': {
        "class": "logging.StreamHandler",
        "formatter": "simple",
        "stream": "ext://sys.stdout"\
        }
    },
"loggers": {
    "root": {"level": "DEBUG", "handlers": ["stdout"]}
    }
}
```

Tip #6: 
Store the configuration file in YAML or JSON format. 

Tip #6: 
Use ISO-8601 Timestamps format to log in dates. 
```
"formatters": { 
    "detailed": {
        "format": "[%(levelname)s|%(module)s|L%(lineno)d] %(asctime): %(message)", 
        "datefmt" : "%Y-%m-%dT%H:%M:%S%z"
    }
}
```

Tip #7: 
Store persistent logs in JSON format (so they can be parsed easily later on). 

Tip #10: 
For library code do not configure logging. You can still use logging (create loggers, log messages and other important events), just don't configure it with dictConfig and any other config. Let applications do the configuring. If the user does not configure logging the _default_ and _expected_ behavior is that warnings and above will be printed to __standard error__. 

If the user of the library does configure logging then do not interfere with what they want by adding handlers, formatter or other things of which they are not aware of. 




ChatGPT gave the following

In [ ]:
# logging_config.py
import logging

def configure_logging(level=logging.INFO):
    """
    Configures logging for the application.
    This function should be called from the main application entry point.
    """
    # Set up the root logger
    logging.basicConfig(level=level,
                        format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

def get_logger(name):
    """
    Creates and returns a logger for a given module.
    This ensures consistent logging across the library.
    """
    return logging.getLogger(name)


In [ ]:
# performance_tracker.py
import time
import psutil
import logging

class PerformanceTracker:
    def __init__(self, logger=None):
        # Use the provided logger, or get a default one
        self.logger = logger or logging.getLogger(__name__)

    def time_and_track(self, log=True, to_print=True):
        """
        Decorator for timing and resource usage (CPU, Memory).
        """
        def decorator(func):
            def wrapper(*args, **kwargs):
                # Record the start time and resource usage
                start_time = time.perf_counter()
                start_cpu = psutil.cpu_percent(interval=None)
                start_memory = psutil.Process().memory_info().rss / (1024 * 1024)  # Memory in MB
                
                # Execute the wrapped function
                result = func(*args, **kwargs)
                
                # Record the end time and resource usage
                end_time = time.perf_counter()
                end_cpu = psutil.cpu_percent(interval=None)
                end_memory = psutil.Process().memory_info().rss / (1024 * 1024)  # Memory in MB
                
                # Calculate time taken, CPU, and memory usage
                time_taken = end_time - start_time
                cpu_usage = end_cpu - start_cpu
                memory_usage = end_memory - start_memory
                
                message = (f"Function '{func.__name__}' executed in {time_taken:.4f} seconds, "
                           f"CPU usage: {cpu_usage:.2f}%, "
                           f"Memory usage change: {memory_usage:.2f} MB.")

                # Log and/or print the message
                if log:
                    self.logger.info(message)
                if to_print:
                    print(message)

                return result
            return wrapper
        return decorator


In [ ]:
# module1.py
from logging_config import get_logger
from performance_tracker import PerformanceTracker

logger = get_logger(__name__)
tracker = PerformanceTracker(logger)

@tracker.time_and_track()
def some_function_in_module1():
    # Function logic
    for _ in range(1000000):
        pass

In [ ]:
# main.py (the application entry point)
import logging_config
import module1
import module2

if __name__ == "__main__":
    # Configure logging at the application level
    logging_config.configure_logging(logging.DEBUG)
    
    # Call functions from modules
    module1.some_function_in_module1()

In [ ]:
import time
import functools

def conditional_decorator(condition, decorator):
    """
    A factory that returns a conditional decorator.
    If `condition` is True, applies `decorator`. Otherwise, returns the original function.
    
    Parameters:
    - condition (bool): The condition to check.
    - decorator (function): The decorator to apply if the condition is True.
    """
    def wrapper(func):
        # If the condition is true, apply the decorator
        if condition:
            return decorator(func)
        # Otherwise, return the original function
        return func
    return wrapper

# Timing decorator
def timeit(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        time_taken = end_time - start_time
        print(f"Function '{func.__name__}' executed in {time_taken:.4f} seconds.")
        return result
    return wrapper

# Example function
@conditional_decorator(condition=True, decorator=timeit)
def some_function():
    for _ in range(1000):
        pass
    print("Func with timing")

# Example function with condition=False (no timing)
@conditional_decorator(condition=False, decorator=timeit)
def another_function():
    for _ in range(1000):
        pass
    print("Func without timing")

# Test the functions
some_function()  # This will be timed
another_function()  # This will run without timing

In [ ]:
import time
import psutil
import logging
import functools

In [ ]:
class RGBLogger:
    def __init__(self, log_config = None): 
        pass 

    @staticmethod
    def rgb_decorated(decorate, decorator, logger):
    
        def wrapper(func): 
            if decorate and logger is not None: 
                return decorator(func, logger) 
            return func
            
        return wrapper
    
    @staticmethod 
    def track_perf(func, logger):
    
        @functools.wraps(func)
        def wrapper(*args, **kwargs):

            start_time = time.perf_counter()
            logger.info(f"Started execution of `{func.__name__}`")
            
            result = func(*args, **kwargs)
    
            end_time = time.perf_counter()
            time_taken = end_time - start_time
            logger.info(f"Finished execution of `{func.__name__}`: {time_taken:.2f}s")
            
            return result
    
        return wrapper 


In [ ]:
#@trackit
def divide(a, b): 
    return a / b

In [ ]:
a = 2
b = 1

logging.basicConfig(level=logging.DEBUG, format="%(levelname)s - %(name)s - %(message)s")
logger = logging.getLogger(__name__)

#logging.basicConfig(level=logging.DEBUG, filemode="w", encoding ="utf-8", format="%(levelname)-%(name)-%(messages)s") 

def logged(a, b, logger = None): 

    tracked = RGBLogger.rgb_decorated(decorate = True, decorator = RGBLogger.track_perf, logger = logger)(divide)
    result = tracked(a, b)

    return result 

#result = divide(a,b)
print(logged(a, b, logger = logger))